In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from lazypredict.Supervised import LazyClassifier

# Fonction générique pour afficher un résumé des colonnes
def decrire_colonnes(df):
    return pd.DataFrame({
        "Nom de la colonne": df.columns,
        "Type de données": df.dtypes.values,
        "Valeurs manquantes": df.isna().sum().values,
        "Pourcentage de valeurs manquantes": (df.isna().mean() * 100).values,
        "Valeurs uniques": df.nunique().values
    })

# Fonction pour générer un catalogue de données
def generer_catalogue(df):
    print("\n===== DESCRIPTION DES COLONNES =====")
    decrire_colonnes(df)
    print("\n===== STATISTIQUES DESCRIPTIVES =====")
    df.describe()

# Fonction générique pour visualiser la distribution des variables numériques
def visualiser_distributions(df, bins=30):
    num_cols = df.select_dtypes(include=[np.number]).columns
    df[num_cols].hist(figsize=(12, 8), bins=bins, edgecolor='black')
    plt.suptitle("Distribution des caractéristiques", fontsize=14)
    plt.show()

# Fonction générique pour visualiser la matrice de corrélation
def visualiser_correlation(df):
    plt.figure(figsize=(10, 6))
    sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
    plt.title("Matrice de corrélation des variables")
    plt.show()

# Fonction générique pour la préparation des données
def preparer_donnees(df, target, drop_cols=[]):
    df = df.dropna()
    X = df.drop(columns=[target] + drop_cols, errors='ignore')
    y = LabelEncoder().fit_transform(df[target])
    X = pd.get_dummies(X, drop_first=True)
    scaler = StandardScaler()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test

# Fonction pour tester plusieurs modèles rapidement avec LazyPredict
def tester_modeles_rapides(X_train, X_test, y_train, y_test):
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)
    return models

# Fonction générique pour entraîner plusieurs modèles
def entrainer_modeles(X_train, X_test, y_train, y_test, models=None):
    if models is None:
        models = {
            "Logistic Regression": LogisticRegression(),
            "KNN": KNeighborsClassifier(n_neighbors=5),
            "Decision Tree": DecisionTreeClassifier(max_depth=5),
            "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=5),
            "SVM": SVC(kernel='linear', probability=True)
        }
    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = classification_report(y_test, y_pred, output_dict=True)["weighted avg"]["f1-score"]
        auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
        results[name] = {"Accuracy": accuracy, "F1 Score": f1, "ROC AUC": auc}
    return results

# Fonction générique pour optimiser les hyperparamètres
def optimiser_hyperparametres(X_train, y_train, param_grid):
    best_models = {}
    for name, obj in param_grid.items():
        grid_search = GridSearchCV(obj["model"], obj["params"], cv=5, scoring="accuracy")
        grid_search.fit(X_train, y_train)
        best_models[name] = {"Best Model": grid_search.best_estimator_, "Best Params": grid_search.best_params_, "Best Score": grid_search.best_score_}
    return best_models

# Sélection générique du meilleur modèle
def selectionner_meilleur_modele(resultats, metric="Accuracy"):
    return max(resultats, key=lambda x: resultats[x][metric])

# Fonction générique pour la prédiction
def faire_prediction(model, X_new):
    return model.predict(X_new)